In [1]:
!pip install agentops

In [2]:
!pip install autogen

In [3]:
import agentops

from autogen import ConversableAgent , UserProxyAgent , config_list_from_json

agentops.init(api_key="...")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=391e3b2d-2c65-4e87-a487-5b3063802e54


In [4]:
#Simple Chat Example
import agentops

agentops.init(tags=["Autogen_Example"]) #this tag is optional just to help filter session

#creating the agent that will use the llm
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST.json")
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

#Creating the agent that will use the user in the conversation
user_proxy = UserProxyAgent("user" , code_execution_config=False)

#In the next process letting the assistant start the process
assistant.initiate_chat(user_proxy, message = "How can i help you")

#closing the agentops session just to indicate that the process is completed
agentops.end_session("Success")

🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.


agent (to user):

How can i help you

--------------------------------------------------------------------------------
Replying as user. Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: who is virat kohli ?
user (to agent):

who is virat kohli ?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent (to user):

Virat Kohli is an Indian cricketer and one of the most prominent figures in international cricket. Born on November 5, 1988, in Delhi, India, Kohli is known for his aggressive batting style, remarkable consistency, and leadership abilities.

He made his debut for the Indian national team in 2008 and quickly established himself as one of the top batsmen in the world. Kohli has set numerous records, including being one of the fastest players to reach various milestones in One Day Internationals (ODIs) and T20s. He has been the captain of the Indian cricket team 

🖇 AgentOps: Session Stats - Duration: 28.7s | Cost: $0.000157 | LLMs: 1 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=391e3b2d-2c65-4e87-a487-5b3063802e54


In [5]:
!pip install typing

In [16]:
#Tool Example
from typing import Annotated , Literal #used for type annotations in Python.
from autogen import ConversableAgent , UserProxyAgent , config_list_from_json , register_function #enabling the creation of agents and function registration for interaction.

agentops.start_session(tags=["Autogen_Example"]) #This line starts a session with agentops

Operator = Literal["+" , "-" , "*" , "/"] #our basic arithmetic operators

def calculator(a:int, b:int, operator: Annotated[Operator, "operator"]) -> int:
  if operator == "+":
    return a+b;
  elif operator == "-":
    return a-b;
  elif operator == "*":
    return a*b;
  elif operator == "/":
    return a/b;
  else:
    return ValueError("Invalid Operator")

config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST.json") #This line loads a configuration list from a JSON file named


#creating the agent that willl use the llm
assistant = ConversableAgent(
    name="assistant",
    system_message="You are a helpful assistant"
    "You can help with simple calculations"
    "Return 'TERMINATE' when task is done",
    llm_config={"config_list" : config_list},
)

#calling the user proxy agent fopr interacting with the assiastant agent
user_proxy =ConversableAgent(
    name="user",
    llm_config=False,
    is_termination_msg=lambda msg:msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode= "NEVER",
)

assistant.register_for_llm(name="calculator", description="A Simple Calculator")(calculator) #Registers calculator function with the assistant for generating responses using LLM
user_proxy.register_for_execution(name="calculator")(calculator) #Registers calculator function with the user proxy to execute it.

#registering the calculator funciton to the two function
register_function(
    calculator,
    caller=assistant,
    executor=user_proxy,
    name="calculator",
    description="A SImple Calculator",
)

#the assitant will start the coversation
user_proxy.initiate_chat(assistant ,  message = "(1520−220)/4+(45+18)∗6?")

agentops.end_session("success")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=05509814-094f-4470-a3af-19f2b2dcdc28


user (to assistant):

(1520−220)/4+(45+18)∗6?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2596: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:2514: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)
🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


assistant (to user):

***** Suggested tool call (call_pJBuJHrceIioa1HkDcd1ZxoQ): calculator *****
Arguments: 
{"a": 1520, "b": 220, "operator": "-"}
***************************************************************************
***** Suggested tool call (call_BZRT66mgcgeMXUyiBs7Ubw7D): calculator *****
Arguments: 
{"a": 45, "b": 18, "operator": "+"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_pJBuJHrceIioa1HkDcd1ZxoQ
Input arguments: {'a': 1520, 'b': 220, 'operator': '-'}

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_BZRT66mgcgeMXUyiBs7Ubw7D
Input arguments: {'a': 45, 'b': 18, 'operator': '+'}
user (to assistant):

***** Response from calling tool (call_pJBuJHrceIioa1HkDcd1ZxoQ) *****
1300
**********************************************************************

--------------------------------------------------

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


...

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


assistant (to user):

***** Suggested tool call (call_FShMwdSCGO4bLyWp1PHnOjr9): calculator *****
Arguments: 
{"a":325,"b":378,"operator":"+"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_FShMwdSCGO4bLyWp1PHnOjr9
Input arguments: {'a': 325, 'b': 378, 'operator': '+'}
user (to assistant):

***** Response from calling tool (call_FShMwdSCGO4bLyWp1PHnOjr9) *****
703
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


assistant (to user):

The final result of the expression (1520−220)/4+(45+18)∗6 is 703.

TERMINATE

--------------------------------------------------------------------------------


🖇 AgentOps: Could not end session - multiple sessions detected. You must use session.end_session() instead of agentops.end_session() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
